In [ ]:
import json
import pandas as pd
from collections import Counter
from nltk.tokenize import word_tokenize

In [ ]:
f=open("test.json")
test_set=json.load(f)

f=open("training.json")
training_set=json.load(f)

In [ ]:
f=open("full_test_set_CRF.json")
test_set_features=json.load(f)

f=open("full_training_set_CRF.json")
training_set_features=json.load(f)

In [ ]:
l_training=[]
for i in range(len(training_set)):
    l_training.append({"text":training_set[i]["data"]["text"], "results":training_set[i]["predictions"][0]["result"]})

l_test=[]
for i in range(len(test_set)):
    l_test.append({"text":test_set[i]["data"]["text"], "results":test_set[i]["predictions"][0]["result"]})

In [ ]:
def eliminate_patient_information(string):
    index = string.find("informe")
    if index != -1:
        return string[index:], index
    else:
        return string, 0

def eliminate_final_information(string):
    index = string.find("destinacio a l'alta")
    if index != -1:
        return string[:index]
    else:
        return string

def nhc_lopd(string):
    nhc_indices = [i for i in range(len(string)) if string.startswith("nhc", i)]
    lopd_indices = [i+len("lopd") for i in range(len(string)) if string.startswith("lopd", i)]

    results = []
    for nhc_index, lopd_index in zip(nhc_indices, lopd_indices):
        if nhc_index < lopd_index:
            length = lopd_index - nhc_index 
            results.append((nhc_index, length))
            break  # break inner loop, move to next nhc_index

    return results

def correct_results(results, lenght, nhclopd):
    for i in range(len(results)):
        results[i]["value"]["start"]-=lenght
        results[i]["value"]["end"]-=lenght
        for j in nhclopd:
            if(j[0]<results[i]["value"]["start"]):
                results[i]["value"]["start"]-=j[1]
                results[i]["value"]["end"]-=j[1]
    return results

def preprocessing(text_dict):
    text=text_dict["text"]
    results=text_dict["results"]
    text, lenght=eliminate_patient_information(text)
    text=eliminate_final_information(text)
    nhclopd=nhc_lopd(text)
    for i in (nhclopd):
        index1=i[0]
        index2=i[0]+i[1]
        text=text[0:index1]+text[index2:]
    results=correct_results(results, lenght, nhclopd)
    text_dict["text"]=text
    text_dict["results"]=results
    return text_dict

In [ ]:
l_training_pro=[]
for i in range(len(l_training)):
    l_training_pro.append(preprocessing(l_training[i]))

l_test_pro=[]
for i in range(len(l_test)):
    l_test_pro.append(preprocessing(l_test[i]))

In [ ]:
def tagger(text, results):
    l_text=word_tokenize(text)
    for i in range(len(l_text)):
        if("``" in l_text[i] or "''" in l_text[i]):
            l_text[i]="\""
    pos=[0,]
    tags=[]
    l_starts=[x["value"]["start"] for x in results]
    l_ends=[x["value"]["end"] for x in results]
    l_labels=[x["value"]["labels"][0] for x in results]
    for i in range(1,len(l_text)):
        j=pos[-1]+len(l_text[i-1])
        if(l_text[i]!="\""):
            while not text.startswith(l_text[i],j):
                j+=1
        pos.append(j)
    for i in range(len(text)):
        if(i in pos):
            if(i in l_starts):
                index=l_starts.index(i)
                label=l_labels[index]
                tags.append(label)
            else:
                x=False
                for j in range(len(l_starts)):
                    if(l_starts[j]<i and i<l_ends[j]):
                        label=l_labels[j]
                        tags.append(label)
                        x=True
                        break
                if not x:
                    tags.append("O")
    return [(word, tag) for word, tag in zip(l_text, tags)]

In [ ]:
for dic in test_set_features:
    dic["text_tagged"]=tagger(dic["text"], dic["results"])

for dic in training_set_features:
    dic["text_tagged"]=tagger(dic["text"], dic["results"])

In [ ]:
for dic1, dic2 in zip(l_test_pro, test_set_features):
    dic1["words"]=dic2["words"]

for dic1, dic2 in zip(l_training_pro, training_set_features):
    dic1["words"]=dic2["words"]

In [ ]:
with open("full_test_set_CRF_tagged.json","w") as f:
  json.dump(l_test_pro,f)

with open("full_training_set_CRF_tagged.json","w") as f:
  json.dump(l_training_pro,f)